In [25]:
import json
import os
from tqdm import tqdm
import re
from transformers import BertTokenizerFast
import copy
import torch
from common.utils import Preprocessor
import yaml
import logging
from pprint import pprint
from IPython.core.debugger import set_trace

In [26]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [27]:
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
config = yaml.load(open("build_data_config.yaml", "r"), Loader = yaml.FullLoader)

In [28]:
exp_name = config["exp_name"]
data_in_dir = os.path.join(config["data_in_dir"], exp_name)
data_out_dir = os.path.join(config["data_out_dir"], exp_name)
if not os.path.exists(data_out_dir):
    os.makedirs(data_out_dir)

# Load Data

In [29]:
file_name2data = {}
for path, folds, files in os.walk(data_in_dir):
    for file_name in files:
        file_path = os.path.join(path, file_name)
        file_name = re.match("(.*?)\.json", file_name).group(1)
        file_name2data[file_name] = json.load(open(file_path, "r", encoding = "utf-8"))

In [30]:
file_name2data.keys()

dict_keys(['huayandan_val_0620', 'huayandan_test_0620', 'huayandan_train_0620'])

# Preprocess

In [31]:
# @specific
if config["encoder"] == "BERT":
    print(config['bert_path'])
    tokenizer = BertTokenizerFast.from_pretrained(config["bert_path"], add_special_tokens = False, do_lower_case = False)
    tokenize = tokenizer.tokenize
    get_tok2char_span_map = lambda text: tokenizer.encode_plus(text, return_offsets_mapping = True, add_special_tokens = False)["offset_mapping"]
elif config["encoder"] == "BiLSTM":
    tokenize = lambda text: text.split(" ")
    def get_tok2char_span_map(text):
        tokens = tokenize(text)
        tok2char_span = []
        char_num = 0
        for tok in tokens:
            tok2char_span.append((char_num, char_num + len(tok)))
            char_num += len(tok) + 1 # +1: whitespace
        return tok2char_span

../pretrained_models/BERT_EMR


In [32]:
preprocessor = Preprocessor(tokenize_func = tokenize, 
                            get_tok2char_span_map_func = get_tok2char_span_map)

## Transform

In [33]:
ori_format = config["ori_data_format"]
if ori_format != "tplinker": # if tplinker, skip transforming
    for file_name, data in file_name2data.items():
        if "train" in file_name:
            data_type = "train"
        if "valid" in file_name:
            data_type = "valid"
        if "test" in file_name:
            data_type = "test"
        data = preprocessor.transform_data(data, ori_format = ori_format, dataset_type = data_type, add_id = True)
        file_name2data[file_name] = data

## Clean and Add Spans

In [34]:
# check token level span
def check_tok_span(data):
    def extr_ent(text, tok_span, tok2char_span):
        char_span_list = tok2char_span[tok_span[0]:tok_span[1]]
        char_span = (char_span_list[0][0], char_span_list[-1][1])
        decoded_ent = text[char_span[0]:char_span[1]]
        return decoded_ent

    span_error_memory = set()
    for sample in tqdm(data, desc = "check tok spans"):
        text = sample["text"]
        tok2char_span = get_tok2char_span_map(text)
        for ent in sample["entity_list"]:
            tok_span = ent["tok_span"]
            if extr_ent(text, tok_span, tok2char_span) != ent["text"]:
                span_error_memory.add("extr ent: {}---gold ent: {}".format(extr_ent(text, tok_span, tok2char_span), ent["text"]))
                
        for rel in sample["relation_list"]:
            subj_tok_span, obj_tok_span = rel["subj_tok_span"], rel["obj_tok_span"]
            if extr_ent(text, subj_tok_span, tok2char_span) != rel["subject"]:
                span_error_memory.add("extr: {}---gold: {}".format(extr_ent(text, subj_tok_span, tok2char_span), rel["subject"]))
            if extr_ent(text, obj_tok_span, tok2char_span) != rel["object"]:
                span_error_memory.add("extr: {}---gold: {}".format(extr_ent(text, obj_tok_span, tok2char_span), rel["object"]))
                
    return span_error_memory

In [35]:
# clean, add char span, tok span
# collect relations
# check tok spans
rel_set = set()
ent_set = set()
error_statistics = {}
for file_name, data in file_name2data.items():
    assert len(data) > 0
    if "relation_list" in data[0]: # train or valid data
        # rm redundant whitespaces
        # separate by whitespaces
        data = preprocessor.clean_data_wo_span(data, separate = config["separate_char_by_white"])
        error_statistics[file_name] = {}
#         if file_name != "train_data":
#             set_trace()
        # add char span
        if config["add_char_span"]:
            data, miss_sample_list = preprocessor.add_char_span(data, config["ignore_subword"])
            error_statistics[file_name]["miss_samples"] = len(miss_sample_list)
            
#         # clean
#         data, bad_samples_w_char_span_error = preprocessor.clean_data_w_span(data)
#         error_statistics[file_name]["char_span_error"] = len(bad_samples_w_char_span_error)
                            
        # collect relation types and entity types
        for sample in tqdm(data, desc = "building relation type set and entity type set"):
            if "entity_list" not in sample: # if "entity_list" not in sample, generate entity list with default type
                ent_list = []
                for rel in sample["relation_list"]:
                    ent_list.append({
                        "text": rel["subject"],
                        "type": "DEFAULT",
                        "char_span": rel["subj_char_span"],
                    })
                    ent_list.append({
                        "text": rel["object"],
                        "type": "DEFAULT",
                        "char_span": rel["obj_char_span"],
                    })
                sample["entity_list"] = ent_list
            
            for ent in sample["entity_list"]:
                ent_set.add(ent["type"])
                
            for rel in sample["relation_list"]:
                rel_set.add(rel["predicate"])
               
        # add tok span
        
        data = preprocessor.add_tok_span(data)

        # check tok span
        if config["check_tok_span"]:
            span_error_memory = check_tok_span(data)
            if len(span_error_memory) > 0:
                print(span_error_memory)
            error_statistics[file_name]["tok_span_error"] = len(span_error_memory)
            
        file_name2data[file_name] = data
pprint(error_statistics)

adding char level spans:  35%|███▍      | 19/55 [00:00<00:00, 185.67it/s]

{'extr: IU/L---gold: U/L', 'extr ent: ALT---gold ent: A', 'extr: ALB---gold: L', 'extr ent: LMID---gold ent: L', 'extr ent: mIU/mL---gold ent: IU/mL', 'extr: TP---gold: P', 'extr: LYM---gold: L', 'extr ent: ALP---gold ent: P', 'extr ent: TP---gold ent: P', 'extr ent: HONH---gold ent: HON', 'extr: min---gold: i', 'extr ent: TBIL---gold ent: L', 'extr: fLe---gold: fL', 'extr ent: CKMB---gold ent: CK', 'extr ent: DBIL---gold ent: L', 'extr: APTT---gold: PT', 'extr ent: ALT---gold ent: L', 'extr: SpO2---gold: S', 'extr: AST---gold: S', 'extr ent: mAST---gold ent: AST', 'extr: GA2---gold: GA', 'extr ent: 糖化血红蛋白Ala---gold ent: 糖化血红蛋白A', 'extr ent: IDBIL---gold ent: L', 'extr: EPi---gold: i', 'extr ent: SpO2---gold ent: S', 'extr ent: EPi---gold ent: P', 'extr ent: IU/L---gold ent: U/L', 'extr: mmol---gold: mmo', 'extr ent: PCT---gold ent: P', 'extr ent: mS/CO---gold ent: S/CO', 'extr: PCT---gold: P', 'extr ent: EPi---gold ent: i', 'extr ent: NEU---gold ent: N', 'extr: fLf---gold: fL', 'extr 

adding char level spans:   8%|▊         | 21/273 [00:00<00:03, 67.19it/s]

{'extr ent: HGB★---gold ent: HGB', 'extr: IU/L---gold: U/L', 'extr: x10 9/L---gold: 10 9/L', 'extr ent: 26-140U---gold ent: 26-140', 'extr: =2o1/153---gold: =2o1/1', 'extr ent: 2001U/ml---gold ent: U/ml', 'extr: MCHC---gold: MCH', 'extr ent: 6g/L---gold ent: g/L', 'extr ent: 0.6-1.1g---gold ent: 0.6-1.1', 'extr ent: TP---gold ent: T', 'extr: L120 110---gold: 120 110', 'extr ent: 4mmol/L---gold ent: mmol/L', 'extr: 40IU/L---gold: IU/L', 'extr: 2mmol/L---gold: mmol/L', 'extr ent: MCHC---gold ent: MCH', 'extr: 3.5-5.2mm---gold: 3.5-5.2', 'extr ent: 17D---gold ent: 17', 'extr ent: /15---gold ent: /1', 'extr: TP---gold: P', 'extr ent: 201U/ml---gold ent: U/ml', 'extr: 7-32U---gold: 7-32', 'extr ent: ALP---gold ent: P', 'extr ent: L4.3--5.8---gold ent: 4.3--5.8', 'extr ent: 1mmol/L---gold ent: mmol/L', 'extr: W11---gold: 11', 'extr ent: 40IU/L---gold ent: U/L', 'extr ent: 245IU/L---gold ent: U/L', 'extr: ChT---gold: T', 'extr ent: 501U/L---gold ent: U/L', 'extr ent: TP---gold ent: P', 'extr:

check tok spans: 100%|██████████| 273/273 [00:00<00:00, 698.49it/s]

{'extr: APTT---gold: TT', 'extr: IU/L---gold: U/L', 'extr ent: 1W1---gold ent: 1', 'extr ent: ALT---gold ent: A', 'extr ent: GLO---gold ent: L', 'extr: TRAB---gold: T', 'extr: pC02---gold: C', 'extr ent: 5-40U---gold ent: 5-40', 'extr ent: eL---gold ent: L', 'extr: 1L---gold: 1', 'extr ent: ag/L---gold ent: g/L', 'extr: ALB---gold: L', 'extr: MCHC---gold: MCH', 'extr: 31Apok/ApeB---gold: Apok/ApeB', 'extr ent: LMID---gold ent: L', 'extr ent: mIU/mL---gold ent: IU/mL', 'extr ent: NAPTT---gold ent: APTT', 'extr: 138U/L---gold: U/L', 'extr: ALP---gold: L', 'extr: 2mmol/L---gold: mmol/L', 'extr ent: 4-20U---gold ent: 4-20', 'extr ent: MCHC---gold ent: MCH', 'extr: APTTR---gold: TT', 'extr: TP---gold: P', 'extr: LYM---gold: L', 'extr ent: 2LP---gold ent: P', 'extr ent: ALP---gold ent: P', 'extr ent: DL---gold ent: L', 'extr: GLO---gold: L', 'extr ent: APOuL1---gold ent: P', 'extr ent: LiP---gold ent: P', 'extr ent: NAPTT---gold ent: PT', 'extr: Ca---gold: C', 'extr: 120-230U---gold: 120-230

# Output to Disk

In [36]:
rel_set = sorted(rel_set)
rel2id = {rel:ind for ind, rel in enumerate(rel_set)}

ent_set = sorted(ent_set)
ent2id = {ent:ind for ind, ent in enumerate(ent_set)}

data_statistics = {
    "relation_type_num": len(rel2id),
    "entity_type_num": len(ent2id),
}

for file_name, data in file_name2data.items():
    data_path = os.path.join(data_out_dir, "{}.json".format(file_name))
    json.dump(data, open(data_path, "w", encoding = "utf-8"), ensure_ascii = False)
    logging.info("{} is output to {}".format(file_name, data_path))
    data_statistics[file_name] = len(data)

rel2id_path = os.path.join(data_out_dir, "rel2id.json")
json.dump(rel2id, open(rel2id_path, "w", encoding = "utf-8"), ensure_ascii = False)
logging.info("rel2id is output to {}".format(rel2id_path))

ent2id_path = os.path.join(data_out_dir, "ent2id.json")
json.dump(ent2id, open(ent2id_path, "w", encoding = "utf-8"), ensure_ascii = False)
logging.info("ent2id is output to {}".format(ent2id_path))



data_statistics_path = os.path.join(data_out_dir, "data_statistics.txt")
json.dump(data_statistics, open(data_statistics_path, "w", encoding = "utf-8"), ensure_ascii = False, indent = 4)
logging.info("data_statistics is output to {}".format(data_statistics_path)) 

pprint(data_statistics)

INFO:root:huayandan_val_0620 is output to ../data4bert/huayandan3/huayandan_val_0620.json
INFO:root:huayandan_test_0620 is output to ../data4bert/huayandan3/huayandan_test_0620.json
INFO:root:huayandan_train_0620 is output to ../data4bert/huayandan3/huayandan_train_0620.json
INFO:root:rel2id is output to ../data4bert/huayandan3/rel2id.json
INFO:root:ent2id is output to ../data4bert/huayandan3/ent2id.json
INFO:root:data_statistics is output to ../data4bert/huayandan3/data_statistics.txt


{'entity_type_num': 16,
 'huayandan_test_0620': 55,
 'huayandan_train_0620': 273,
 'huayandan_val_0620': 69,
 'relation_type_num': 9}


# Genrate WordDict

In [37]:
if config["encoder"] in {"BiLSTM", }:
    all_data = []
    for data in list(file_name2data.values()):
        all_data.extend(data)
        
    token2num = {}
    for sample in tqdm(all_data, desc = "Tokenizing"):
        text = sample['text']
        for tok in tokenize(text):
            token2num[tok] = token2num.get(tok, 0) + 1
    
    token2num = dict(sorted(token2num.items(), key = lambda x: x[1], reverse = True))
    max_token_num = 50000
    token_set = set()
    for tok, num in tqdm(token2num.items(), desc = "Filter uncommon words"):
        if num < 3: # filter words with a frequency of less than 3
            continue
        token_set.add(tok)
        if len(token_set) == max_token_num:
            break
        
    token2idx = {tok:idx + 2 for idx, tok in enumerate(sorted(token_set))}
    token2idx["<PAD>"] = 0
    token2idx["<UNK>"] = 1
#     idx2token = {idx:tok for tok, idx in token2idx.items()}
    
    dict_path = os.path.join(data_out_dir, "token2idx.json")
    json.dump(token2idx, open(dict_path, "w", encoding = "utf-8"), ensure_ascii = False, indent = 4)
    logging.info("token2idx is output to {}, total token num: {}".format(dict_path, len(token2idx))) 